In [6]:
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim

In [7]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
content = BeautifulSoup(requests.get(url).content, 'lxml')

In [3]:
table = content.find('table')
td = table.find_all('td')
postcode = []
borough = []
neighbourhood = []

In [10]:
for i in range(0, len(td), 3):
    postcode.append(td[i].text.strip())
    borough.append(td[i+1].text.strip())
    neighbourhood.append(td[i+2].text.strip())

df = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df.columns = ['Postal Code', 'Borough', 'Neighborhood']

df['Borough'].replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=['Borough'], inplace=True)

df['Neighborhood'].replace('Not assigned', "Queen's Park", inplace=True)


df = df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.columns = ['Postal Code', 'Borough', 'Neighborhood']


In [15]:
#coordinates of each postal code

url="http://cocl.us/Geospatial_data"
geospatial_df = pd.read_csv(url)
geospatial_df.columns = ['Postal Code', 'Latitude', 'Longitude']
TO_df = pd.merge(df, geospatial_df, on=['Postal Code'], how='inner')
TO_df .head(15)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge, Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn, Woburn, Woburn",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae, Cedarbrae, Cedarbrae",43.773136,-79.239476
5,M1J,Scarborough,"Scarborough Village, Scarborough Village, Scar...",43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park, K...",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge, Golden Mile, ...",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village Wes...",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West, Birch Cliff, Clif...",43.692657,-79.264848


In [13]:
#getting lattitude and longitude of Toronto

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="coursera-capstone-project")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


In [14]:
#Boroughs containing TORONTO

TO_data = TO_df[TO_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
TO_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,"The Beaches, The Beaches, The Beaches",43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale, The Danforth Wes...",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West, India Bazaar, ...",43.668999,-79.315572
3,M4M,East Toronto,"Studio District, Studio District, Studio District",43.659526,-79.340923
4,M4N,Central Toronto,"Lawrence Park, Lawrence Park, Lawrence Park",43.728020,-79.388790
